In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import random
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 1. Ler o dataset
df = pd.read_csv("C:\\Users\\carlo\\OneDrive - Danmarks Tekniske Universitet\\1st Year 1st Semester\\02452 Machine Learning\\project1-ml\\data\\ObesityDataSet_Clean.csv")

categorical_cols = ['Gender', 'SMOKE', 'family_history_with_overweight', 'CAEC', 'MTRANS', 'NObeyesdad']

binary_cols = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
for col in binary_cols:
        X[col] = X[col].map({'yes': 1, 'no': 0})

freq_cols = ['CAEC', 'CALC']


X = pd.get_dummies(X, columns=['MTRANS'], drop_first=True)
X = pd.get_dummies(X, columns=['CAEC', 'CALC'], drop_first=True)


obesity_order = ["Insufficient_Weight","Normal_Weight","Overweight_Level_I","Overweight_Level_II","Obesity_Type_I","Obesity_Type_II","Obesity_Type_III"]
X["NObeyesdad"] = X["NObeyesdad"].apply(lambda x: obesity_order.index(x) + 1)

#feature_cols = ["Age", "Height", "FCVC", "NCP", "CH2O", "FAF", "TUE"]

# define target and features (just to extract y)
y = X["Weight"]
X = X.drop(columns=["Weight"])

# baseline ignores X, but we still need to split the same way
print(X.dtypes)
X.head()


KeyError: "None of [Index(['CAEC', 'CALC'], dtype='object')] are in the [columns]"

*FIND BEST SET OF H (ANN)*

In [18]:
def get_model(input_dim, hidden_dim, output_dim):
    ### BEGIN SOLUTION
    return torch.nn.Sequential(
        torch.nn.Linear(in_features=input_dim, out_features=hidden_dim, bias=True),     # Input layer
        torch.nn.Tanh(),                                                                # Activation function
        torch.nn.Linear(in_features=hidden_dim, out_features=output_dim, bias=True),    # Output layer
    )

mean, std = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mean) / std
X_val = (X_val - mean) / std


# K-fold crossvalidation
K = 3
CV = KFold(K, shuffle=True, random_state=0)

# Define hyperparameters
lr = 1e-3
n_epochs = 1000

# Seed for reproducibility
seed = 0

# Hyperparameter tuning loop with K-fold crossvalidation
hyperparameters_to_tune = [1, 2, 3, 10, 15, 50]

results = {}
for k, (train_index, val_index) in enumerate(CV.split(X, y)):
    print(f'Fold {k+1}/{K}')
    
    # Get the training and test data for this fold
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Normalize data here
    mean, std = X_train.mean(axis=0), X_train.std(axis=0)
    X_train = (X_train - mean) / std
    X_val = (X_val - mean) / std

    # Convert to torch tensors
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

    input_dim  = X_train_tensor.shape[1]  # Number of features
    hidden_dim = 2
    output_dim = 1


    # Set up a dictionary to store the results for each hyperparameter setting
    results_inner = {hidden_dim: {'train': [], 'val': []} for hidden_dim in hyperparameters_to_tune}

    # Loop over the hyperparameter settings        
    for hidden_dim in hyperparameters_to_tune:
        # Define a model instance with a specific number of hidden units
        model = get_model(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

        # Define loss criterion
        criterion = torch.nn.MSELoss()

        # Define the optimizer as the Adam optimizer (not needed to know the details)
        optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
        
        for epoch in range(n_epochs):
            # Implement the training loop here
            # Set the model to training mode
            model.train()

            # Make a forward pass through the model to compute the outputs
            outputs = model(X_train_tensor)
            # Compute the loss
            loss = criterion(outputs, y_train_tensor)

            # Make sure that the gradients are zero before you use backpropagation
            optimizer.zero_grad()
            # Do a backward pass to compute the gradients wrt. model parameters using backpropagation.
            loss.backward()
            # Update the model parameters by making the optimizer take a gradient descent step
            optimizer.step()

            # Store the training loss for this epoch in the dictionary
            results_inner[hidden_dim]['train'].append(loss.item())

        # Compute the final test loss on the test set
        with torch.no_grad(): # No need to compute gradients for the validation set
            model.eval()
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor)
            results_inner[hidden_dim]['val'].append(val_loss.item())
            print(f'  Hidden units: {hidden_dim}, Validation set MSE: {val_loss.item():.4f}')

    # Store the results for this fold
    results[k] = results_inner


# Plot the loss curves for each fold and hyperparameter setting
fig, axs = plt.subplots(1, K, figsize=(12, 4), sharey=True, sharex=True)
# Plot the training loss for each fold and hyperparameter setting
for fold in range(K):
    for hidden_dim in hyperparameters_to_tune:
        # Plot the training loss for this hyperparameter setting
        axs[fold].plot(results[fold][hidden_dim]['train'], label=f'hidden_dim={hidden_dim}')

    # Set the title and labels for each subplot
    axs[fold].set_title('Fold {}'.format(fold+1))
    axs[fold].set_xlabel('Epoch')
    axs[fold].set_ylabel('MSE')

# Set the overall title and show the legend
plt.suptitle('Training loss for different hidden units')
plt.tight_layout()
axs[0].legend()
plt.show()

print(y.min(), y.max(), y.mean())


Fold 1/3


TypeError: Could not convert ['nonononoyesyesnoyesyesnoyesyesyesnoyesyesyesnononoyesyesyesnoyesyesnoyesyesyesyesyesyesyesyesyesyesyesyesnonononoyesnoyesnoyesnoyesyesyesnononoyesnononononoyesnoyesyesyesyesyesnoyesyesyesyesyesyesyesnoyesyesyesyesyesyesyesyesyesnoyesyesyesnonoyesyesyesnonoyesyesyesnoyesyesnononononononoyesyesnonoyesyesyesyesyesyesyesyesnonoyesyesnoyesyesyesyesnoyesnoyesyesyesyesnoyesnonoyesyesyesnoyesnoyesyesnoyesnoyesyesyesyesyesyesyesnoyesyesyesyesnononononoyesyesyesyesyesyesnoyesyesyesyesyesnoyesyesyesyesyesyesyesyesnonoyesnoyesnoyesyesyesnoyesyesnoyesnoyesyesyesnonoyesyesyesyesyesyesyesyesnoyesnonoyesyesnoyesyesnoyesyesyesnoyesnonoyesyesyesyesyesyesnoyesyesyesyesyesyesnoyesnonoyesnoyesnoyesyesnoyesyesnoyesnoyesnoyesyesnononoyesyesyesyesyesyesyesyesnonoyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesyesnoyesyesyesyesyesnonoyesyesyesyesyesyesnoyesyesyesyesyesyesyesyesyesyesyesyesyesnoyesyesyesyesyesyesyesyesyesnononoyesyesyesyesyesnoyesyesyesyesyesyesnononoyesyesyesyesy

*TWO LAYER CROSS VALIDATION*

In [11]:

def cross_validation(X, y, outer_folds=10, inner_folds=10, random_state=42):

    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    lambda_grid = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
    rows = []
    CV_outer = KFold(n_splits=outer_folds, shuffle=True, random_state=random_state)
    outer_test_mse = []
    CV_inner = KFold(n_splits=inner_folds, shuffle=True, random_state=random_state)
    inner_mse = []


    for outer_fold_idx, (outer_train_idx, outer_test_idx) in enumerate(CV_outer.split(X), start=1):
        X_train_outer, X_test_outer = X.iloc[outer_train_idx], X.iloc[outer_test_idx]
        y_train_outer, y_test_outer = y.iloc[outer_train_idx], y.iloc[outer_test_idx]

        #standerdize data 
        mean, std = X_train_outer.mean(axis=0), X_train_outer.std(axis=0)
        X_train_outer = (X_train_outer - mean) / std
        X_test_outer = (X_test_outer - mean) / std

        lambda_errors = {}
        for lam in lambda_grid:
            inner_mses = []
            for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer):
                X_train_inner = X_train_outer.iloc[inner_train_idx]
                X_test_inner = X_train_outer.iloc[inner_test_idx]
                y_train_inner = y_train_outer.iloc[inner_train_idx]
                y_test_inner = y_train_outer.iloc[inner_test_idx]

                # LINEAR REGRESSION MODEL
                model = Ridge(alpha=lam)
                model.fit(X_train_inner, y_train_inner)
                y_pred_inner = model.predict(X_test_inner)
                inner_mse.append(mean_squared_error(y_test_inner, y_pred_inner))
                E_test_lin = mean_squared_error(y_test_outer,  model.predict(X_test_outer))
                inner_mses.append(mean_squared_error(y_test_inner, y_pred_inner))
                
            lambda_errors[lam] = np.mean(inner_mses)

        best_lambda = min(lambda_errors, key=lambda_errors.get)

        # TRAIN FINAL MODEL ON FULL TRAINING SET WITH BEST LAMBDA
        model = Ridge(alpha=best_lambda)
        model.fit(X_train_outer, y_train_outer)
        E_test_lin = mean_squared_error(y_test_outer, model.predict(X_test_outer))
        outer_test_mse.append(E_test_lin)

        # BASELINE MODEL
        baseline_pred = float(np.mean(y_train_outer))       
        E_test_base = mean_squared_error(y_test_outer, np.full(len(y_test_outer), baseline_pred))

        # ANN MODEL 
        lr = 0.01
        n_epochs = 1000 
        input_dim  = X_train_outer.shape[1]  # Number of features
        hidden_dim = 2
        output_dim = 1
        hyperparameters_to_tune = [1, 2, 3, 4, 5]
        best_h, best_val = None, np.inf
        criterion = torch.nn.MSELoss()

        def get_model(input_dim, hidden_dim, output_dim):
            return torch.nn.Sequential(
                torch.nn.Linear(in_features=input_dim, out_features=hidden_dim, bias=True),     # Input layer
                torch.nn.Tanh(),                                                                # Activation function
                torch.nn.Linear(in_features=hidden_dim, out_features=output_dim, bias=True),    # Output layer
            )

        # Loop over the hyperparameter settings        
        for hidden_dim in hyperparameters_to_tune:
            fold_vals = []
            for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer):
                X_train_inner = X_train_outer.iloc[inner_train_idx]
                X_test_inner = X_train_outer.iloc[inner_test_idx]
                y_train_inner = y_train_outer.iloc[inner_train_idx]
                y_test_inner = y_train_outer.iloc[inner_test_idx]

                X_train_inner_tensor = torch.tensor(X_train_inner.values, dtype=torch.float32)
                y_train_inner_tensor = torch.tensor(y_train_inner.values, dtype=torch.float32).view(-1, 1)
                X_test_inner_tensor  = torch.tensor(X_test_inner.values, dtype=torch.float32)
                y_test_inner_tensor  = torch.tensor(y_test_inner.values, dtype=torch.float32).view(-1, 1)
            
                model = get_model(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
                optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
            
                for _ in range(n_epochs):
                    model.train()
                    outputs = model(X_train_inner_tensor)
                    loss = criterion(outputs, y_train_inner_tensor)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                with torch.no_grad():
                    model.eval()
                    fold_vals.append(criterion(model(X_test_inner_tensor), y_test_inner_tensor).item())

            val_loss = np.mean(fold_vals)
            if val_loss < best_val:
                best_val = val_loss
                best_h = hidden_dim

        # TRAIN FINAL MODEL ON FULL TRAINING SET WITH BEST HYPERPARAMETER
        X_train_outer_tensor = torch.tensor(X_train_outer.values.astype(np.float32))
        y_train_outer_tensor = torch.tensor(y_train_outer.values.astype(np.float32).reshape(-1, 1))
        X_test_outer_tensor  = torch.tensor(X_test_outer.values.astype(np.float32))

        model = get_model(input_dim, best_h, output_dim)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        for _ in range(n_epochs):
            model.train()
            optimizer.zero_grad()
            loss = criterion(model(X_train_outer_tensor), y_train_outer_tensor)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            model.eval()
            E_test_ann = mean_squared_error(y_test_outer.values, model(X_test_outer_tensor).detach().numpy().ravel())

        rows.append({"Outer fold": outer_fold_idx,  "λ*_i": best_lambda, "E_test^lin": E_test_lin, "E_test^base": E_test_base,  "h*_i": best_h, "E_test^ann": E_test_ann})

    results = pd.DataFrame(rows).sort_values("Outer fold").reset_index(drop=True)
    print(results.round({"E_test^lin":2, "E_test^base":2, "E_test^ann":2}))
    print("\nAverages (MSE ↓) across outer folds:")
    print(f"Linear regression: {results['E_test^lin'].mean():.2f}")
    print(f"Baseline:          {results['E_test^base'].mean():.2f}")
    print(f"Neural Network:    {results['E_test^ann'].mean():.2f}")

    return results
results  = cross_validation(X, y, outer_folds=10, inner_folds=10, random_state=42)


   Outer fold  λ*_i  E_test^lin  E_test^base  h*_i  E_test^ann
0           1    10      462.83       732.32     5      347.48
1           2    10      439.68       677.76     5      368.58
2           3    10      477.38       736.37     5      413.90
3           4    10      370.21       614.22     5      350.27
4           5    10      460.64       634.78     5      386.83
5           6    10      475.32       685.18     5      414.28
6           7    10      409.38       614.44     5      334.87
7           8    10      424.39       716.06     5      348.40
8           9    10      526.75       786.46     5      382.86
9          10    10      452.80       667.27     5      354.89

Averages (MSE ↓) across outer folds:
Linear regression: 449.94
Baseline:          686.49
Neural Network:    370.24


*STATS EVALUATION*

In [32]:
from scipy import stats

# example: linear vs baseline
d_lin_base = results["E_test^lin"] - results["E_test^base"]
t_stat, p_val = stats.ttest_rel(results["E_test^lin"], results["E_test^base"])

mean_diff = d_lin_base.mean()
std_diff  = d_lin_base.std(ddof=1)
n = len(d_lin_base)
ci_low  = mean_diff - 2.262 * std_diff / np.sqrt(n)
ci_high = mean_diff + 2.262 * std_diff / np.sqrt(n)

print(f"Linear vs Baseline:")
print(f"mean diff = {mean_diff:.3f}")
print(f"t = {t_stat:.3f}, p = {p_val}")
print(f"95% CI = [{ci_low:.3f}, {ci_high:.3f}]")

Linear vs Baseline:
mean diff = -236.551
t = -21.103, p = 5.651823060668685e-09
95% CI = [-261.907, -211.196]


In [37]:
from scipy import stats

# example: linear vs ann
d_lin_base = results["E_test^ann"] - results["E_test^lin"]
t_stat, p_val = stats.ttest_rel(results["E_test^ann"], results["E_test^lin"])

mean_diff = d_lin_base.mean()
std_diff  = d_lin_base.std(ddof=1)
n = len(d_lin_base)
ci_low  = mean_diff - 2.262 * std_diff / np.sqrt(n)
ci_high = mean_diff + 2.262 * std_diff / np.sqrt(n)

print(f"Linear vs ANN:")
print(f"mean diff = {mean_diff:.3f}")
print(f"t = {t_stat:.3f}, p = {p_val}")
print(f"95% CI = [{ci_low:.3f}, {ci_high:.3f}]")

Linear vs ANN:
mean diff = -83.461
t = -7.341, p = 4.3670510870121584e-05
95% CI = [-109.176, -57.745]


In [34]:
from scipy import stats

# example: ann vs baseline
d_ann_base = results["E_test^ann"] - results["E_test^base"]
t_stat, p_val = stats.ttest_rel(results["E_test^ann"], results["E_test^base"])

mean_diff = d_lin_base.mean()
std_diff  = d_lin_base.std(ddof=1)
n = len(d_lin_base)
ci_low  = mean_diff - 2.262 * std_diff / np.sqrt(n)
ci_high = mean_diff + 2.262 * std_diff / np.sqrt(n)

print(f"Linear vs ANN:")
print(f"mean diff = {mean_diff:.3f}")
print(f"t = {t_stat:.3f}, p = {p_val}")
print(f"95% CI = [{ci_low:.3f}, {ci_high:.3f}]")

Linear vs ANN:
mean diff = 83.461
t = -18.961, p = 1.4529217140698072e-08
95% CI = [57.745, 109.176]
